# Scratch pad for test code

In [2]:
import pandas as pd

In [5]:
pd.read_parquet("../ds_feature_store/feature_repo/data/driver_stats.parquet")

,event_timestamp,driver_id,conv_rate,acc_rate,avg_daily_trips,created
0,2025-04-12 23:00:00+00:00,1005,0.137861,0.417238,382,2025-04-27 23:54:57.961
1,2025-04-13 00:00:00+00:00,1005,0.526658,0.001163,49,2025-04-27 23:54:57.961
2,2025-04-13 01:00:00+00:00,1005,0.156601,0.047198,325,2025-04-27 23:54:57.961
3,2025-04-13 02:00:00+00:00,1005,0.967302,0.388253,877,2025-04-27 23:54:57.961
4,2025-04-13 03:00:00+00:00,1005,0.549147,0.632446,260,2025-04-27 23:54:57.961
...,...,...,...,...,...,...
1802,2025-04-27 21:00:00+00:00,1001,0.447199,0.105152,537,2025-04-27 23:54:57.961
1803,2025-04-27 22:00:00+00:00,1001,0.232437,0.692605,677,2025-04-27 23:54:57.961
1804,2021-04-12 07:00:00+00:00,1001,0.357037,0.548742,283,2025-04-27 23:54:57.961
1805,2025-04-20 11:00:00+00:00,1003,0.312929,0.347996,955,2025-04-27 23:54:57.961


In [6]:
import mlflow

In [7]:
mlflow.set_tracking_uri(uri="http://127.0.0.7:8000")

In [11]:
import mlflow
from mlflow.models import infer_signature

import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


# Load the Iris dataset
X, y = datasets.load_iris(return_X_y=True)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Define the model hyperparameters
params = {
    "solver": "lbfgs",
    "max_iter": 1000,
    "random_state": 8888,
}

# Train the model
lr = LogisticRegression(**params)
lr.fit(X_train, y_train)

# Predict on the test set
y_pred = lr.predict(X_test)

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)

In [12]:
# Set our tracking server uri for logging
mlflow.set_tracking_uri(uri="http://127.0.0.7:8080")

# Create a new MLflow Experiment
mlflow.set_experiment("MLflow Quickstart")

# Start an MLflow run
with mlflow.start_run():
    # Log the hyperparameters
    mlflow.log_params(params)

    # Log the loss metric
    mlflow.log_metric("accuracy", accuracy)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", "Basic LR model for iris data")

    # Infer the model signature
    signature = infer_signature(X_train, lr.predict(X_train))

    # Log the model
    model_info = mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path="iris_model",
        signature=signature,
        input_example=X_train,
        registered_model_name="tracking-quickstart",
    )

MlflowException: API request to http://127.0.0.7:8080/api/2.0/mlflow/experiments/get-by-name failed with exception HTTPConnectionPool(host='127.0.0.7', port=8080): Max retries exceeded with url: /api/2.0/mlflow/experiments/get-by-name?experiment_name=MLflow+Quickstart (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f946cfd2900>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [ ]:
# Load the model back for predictions as a generic Python Function model
loaded_model = mlflow.pyfunc.load_model(model_info.model_uri)

predictions = loaded_model.predict(X_test)

iris_feature_names = datasets.load_iris().feature_names

result = pd.DataFrame(X_test, columns=iris_feature_names)
result["actual_class"] = y_test
result["predicted_class"] = predictions

result[:4]